In [0]:
from pyspark.sql.functions import lit

In [0]:
%run "./common_functions"

In [0]:
%run "./configuration"

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

## Ingest constructors.json file

##### Read the JSON file using spark datafram reader

In [0]:
constructors_schema = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

In [0]:
constructors_df = spark.read.schema(constructors_schema).json(f"{raw_folder_path}/{v_file_date}/constructors.json")

##### Drop column

In [0]:
constructors_dropped_df = constructors_df.drop("url")

##### Rename and ingestion date

In [0]:
constructors_final_df = constructors_dropped_df\
    .withColumnRenamed("constructorId", "constructor_id")\
    .withColumnRenamed("constructorRef", "constructor_ref") \
    .withColumn("data_source", lit(v_data_source)) \
    .withColumn("file_date", lit(v_file_date))

In [0]:
constructors_final_df = add_ingestion_date(constructors_final_df)

#### Write output to parquet file

In [0]:
constructors_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.constructors")

In [0]:
dbutils.notebook.exit("Success")